# First Filter: select 3 cities  🌆🌇

<img width=500 src=https://assets.hongkiat.com/uploads/creative-world-maps/10-creative-world-map-remake.jpg>

### Importing libraries  and creating connection with Mongo DB

In [1]:
import re
import pandas as pd
import numpy as np
import src.dataframes as f
from pymongo import MongoClient
client = MongoClient("localhost:27017")
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import src.maps as m
import src.cities_coordinates as cc

In [2]:
db = client.get_database("ironhack")

**"companies" collection has information on 18.8K companies all over the world 🌎🌍🌏**

In [3]:
companies = db.get_collection("companies")

### Doing some queries to filter

#### We want to keep those cities with: 
* Many design companies
* Tech startups that have raised at least 1 Million dollars

In [4]:
query_1 = {'$and' : [{"tag_list" : {'$regex':'design'}}, {"tag_list" : {'$regex':'(?s)^((?!fashion).)*'}},
                     {"tag_list" : {'$regex':'(?s)^((?!interior).)*'}}]}
query_2 = {'$and' : [{"total_money_raised" : {'$regex':'M'}},{"tag_list" : {'$regex':'tech'}}]}
proj = {'name': 1, '_id': 0, 'total_money_raised' : 1, 'offices.city' : 1, 'offices.latitude': 1, 'offices.longitude': 1}
companies_filtered = list(companies.find({'$or' : [query_2, query_1]}, proj))

In [5]:
companies_filtered[0]

{'name': 'Digg',
 'total_money_raised': '$45M',
 'offices': [{'city': 'San Francisco',
   'latitude': 37.764726,
   'longitude': -122.394523}]}

### Creating a Dataframe with the companies filtered

In [6]:
companies_df = f.new_df(companies_filtered)

In [7]:
companies_df.head()

,company,city,latitude,longitude
0,Digg,San Francisco,37.764726,-122.394523
1,Powerset,San Francisco,37.778613,-122.395289
2,CastTV,San Francisco,37.780716,-122.393913
3,eBuddy,Amsterdam,52.364093,4.891946
4,Grockit,San Francisco,37.775196,-122.419204


### Selecting the cities with more companies and creating a subdata frame with them

In [8]:
dict_of_cities = f.vc_to_dict(companies_df.city)

In [9]:
cities = f.subdata(11,dict_of_cities)

In [10]:
cities

['San Francisco', 'New York', 'London']

### 🏆🏆 The selected cities are `San Francisco`, `New York` and `London` 🏆🏆

In [11]:
df = f.create(companies_df,'city',cities)

In [12]:
df.sample(4)

,company,city,latitude,longitude
37,Phreesia,New York,40.739794,-73.985878
414,Novalsys,New York,40.728370,-73.992392
252,Shattered Reality Interactive,San Francisco,37.783046,-122.394067
432,FK3,London,NaN,NaN


### Exporting the dataframe

In [13]:
df.to_csv("data/df.csv", index=False)

## Let's make maps with those cities and their companies!! 🗺 

In [14]:
map_sf = cc.map_city(cc.sf_coordinates())
map_lnd = cc.map_city(cc.london_coordinates())
map_ny = cc.map_city(cc.ny_coordinates())

In [15]:
m.institution(df, 'New York', map_ny)

In [16]:
m.institution(df, 'London', map_lnd)

In [17]:
m.institution(df, 'San Francisco', map_sf)

### Saving the maps

In [18]:
map_lnd.save("maps/map_london.html")

In [19]:
map_sf.save("maps/map_san_francisco.html")

In [20]:
map_ny.save("maps/map_new_york.html")